In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [2]:
# For writing data
data_subdirectory = "Data"
if not os.path.exists(data_subdirectory):
    os.makedirs(data_subdirectory)

# Setting the precision for floating points :
np.set_printoptions(precision=4)

In [3]:
write = 1
eve_presence = 1 #'Random'

num_iter = 2    # To average over
in_range = 200   # Range of steps of initial length
ch_noise = 0.020     # 0.00 - 0.30, 0.05 V/s QBER (eve detection)
in_len = [10*(_+1) for _ in range(in_range)]

out_len = np.empty((num_iter, in_range))    # Length of key output for various initial key sizes
time_taken = np.empty((num_iter, in_range)).astype('object')    # avg time taken by a key of a certain length
QBERs = np.empty((num_iter, in_range))
Eve_detected = np.empty((num_iter, in_range), 'int')    ### Try bool

keys = np.empty((num_iter, in_range))    ### Try a binary representation - could be faster
# SKGR = np.empty(in_range)    # Secure key generation rate

In [ ]:
total_time = time.time()

for J in range(num_iter):    
    print(J)
    for I in range(in_range):
        print(f"{I=}")
        KEY_LENGTH = in_len[I]

        time_taken[J, I] = time.time()
        %run ./Qiskit_rebuilt_2.ipynb
        time_taken[J, I] = time.time() - time_taken[J, I]
        
        out_len[J, I] += len(key)
        keys[J, I] = str(key)
        QBERs[J, I] = QBER
        Eve_detected[J, I] = ((QBER >= 0.25) and eve) + ((QBER < 0.25) and not eve)    # Wheter or not the DETECTION of Eve is CORRECT
        
    print(f"###################################### Iteration {J} complete ######################################")

total_time = time.time() - total_time
print(total_time)

0
I=0
Eve Present!
Key is secure
Eve went unnoticed
I=1
Eve Present!
Key not secure
I=2
Eve Present!
Key is secure
Eve went unnoticed
I=3
Eve Present!
Key is secure
Eve went unnoticed
I=4
Eve Present!
Key not secure
I=5
Eve Present!
Key not secure
I=6
Eve Present!
Key is secure
Eve went unnoticed
I=7
Eve Present!
Key is secure
Eve went unnoticed
I=8
Eve Present!
Key is secure
Eve went unnoticed
I=9
Eve Present!
Key is secure
Eve went unnoticed
I=10
Eve Present!
Key not secure
I=11
Eve Present!
Key not secure
I=12
Eve Present!
Key is secure
Eve went unnoticed
I=13
Eve Present!
Key not secure
I=14
Eve Present!
Key not secure
I=15
Eve Present!
Key not secure
I=16
Eve Present!
Key is secure
Eve went unnoticed
I=17
Eve Present!
Key not secure
I=18
Eve Present!
Key not secure
I=19
Eve Present!
Key not secure
I=20
Eve Present!
Key is secure
Eve went unnoticed
I=21
Eve Present!
Key is secure
Eve went unnoticed
I=22
Eve Present!
Key not secure
I=23
Eve Present!
Key not secure
I=24
Eve Present!


In [ ]:
print(total_time)
elapsed_time = str(int(total_time//3600)) + "h  " + str( int((total_time%3600)//60) ) + "m  " + str(int(total_time%60)) + "s"
elapsed_time

In [ ]:
SKGR = [sum(1 for i in range(num_iter) if QBERs[i, j] < 0.25 and Eve_detected[i, j]) / sum(time_taken[:, j]) for j in range(in_range)]    # Over all iterations
SKGR

In [ ]:
# Open the files to write data to
title = f"\nz = {num_iter}, range = {in_len[0]}-{in_len[-1]}, ch_noise = {ch_noise}, Eve Presence = {eve_presence}, Total Time : {elapsed_time}"

avg_time_taken = [sum([time_taken[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]
avg_out_len = [sum([out_len[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]
avg_QBERs = [sum([QBERs[J, I] for J in range(num_iter)])/num_iter for I in range(in_range)]

In [ ]:
mask = np.isnan(out_len)
print('Number of nan entries : ', len(mask))

nan_indices = np.where(mask)
nan_tuple_np = [(i, j) for (i, j) in zip(nan_indices[0], nan_indices[1])]
nan_tuple_np

In [ ]:
Eve_detected = Eve_detected.astype(int)
Eve_detected

In [ ]:
secure_keys = np.zeros((num_iter, in_range))    ### Try a binary representation - could be faster
secure_keys = keys[(Eve_detected == True)]
secure_keys

In [ ]:
rate = sum(secure_keys == True)
rate

In [ ]:
incorrect = num_iter*in_range - sum(sum(Eve_detected))
detection = f"Eve's detection was incorrect : {incorrect} times (avg = {incorrect/(num_iter*in_range)} ); for QBER Threshold : {0.25 - ch_noise}"
print(detection)

plt.xlabel('initial key length(*100)')
plt.ylabel(f'Times correctly detected Eve(out of {num_iter}')
plt.plot(in_len, sum(Eve_detected))

In [ ]:
False_detections = [(i, j) for (i, j) in zip(np.where(Eve_detected == 0)[0], np.where(Eve_detected == 0)[1])]
# False_detections = np.array(False_detections)
(False_detections[:9])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(12, 8))

# For key length
coefficients_len = np.polyfit(in_len, avg_out_len, 1)
polynomial_len = np.poly1d(coefficients_len)
equation_len = f'y = {coefficients_len[0]:.2f}x + {coefficients_len[1]:.2f}'
ax[0, 0].text(0.05, 0.95, equation_len, transform=ax[0, 0].transAxes, fontsize=10, verticalalignment='top')
ax[0, 0].set_xlabel('Length of initial key')
ax[0, 0].set_ylabel('Length of final key after Information Reconciliation')
ax[0, 0].set_title("Input vs Output key length")
ax[0, 0].plot(in_len, avg_out_len)
ax[0, 0].plot(in_len, polynomial_len(in_len), linestyle='--')
ax[0, 0].minorticks_on()
ax[0, 0].grid(True)

# For QBER
coefficients_qber = np.polyfit(in_len, avg_QBERs, 1)
polynomial_qber = np.poly1d(coefficients_qber)
equation_qber = f'y = {coefficients_qber[0]:.2f}x + {coefficients_qber[1]:.2f}'
ax[0, 1].text(0.05, 0.95, equation_qber, transform=ax[0, 1].transAxes, fontsize=10, verticalalignment='top')
ax[0, 1].set_xlabel('Length of initial key')
ax[0, 1].set_ylabel('Average QBER')
ax[0, 1].set_title("Input length vs QBER")
ax[0, 1].plot(in_len, avg_QBERs)
ax[0, 1].plot(in_len, polynomial_qber(in_len), linestyle='--')
ax[0, 1].minorticks_on()
ax[0, 1].grid(True)

# For time taken for 1 cycle
coefficients_tt = np.polyfit(in_len, avg_time_taken, 1)
polynomial_tt = np.poly1d(coefficients_tt)
equation_tt = f'y = {coefficients_tt[0]:.2f}x + {coefficients_tt[1]:.2f}'
ax[1, 0].text(0.05, 0.95, equation_tt, transform=ax[1, 0].transAxes, fontsize=10, verticalalignment='top')
ax[1, 0].set_xlabel('Length of initial key')
ax[1, 0].set_ylabel('Time taken for 1 cycle')
ax[1, 0].set_title("Input key length vs time taken")
ax[1, 0].plot(in_len, avg_time_taken)
ax[1, 0].plot(in_len, polynomial_tt(in_len), linestyle='--')
ax[1, 0].minorticks_on()
ax[1, 0].grid(True)

# For Secure key generation rate
coefficients_SKGR = np.polyfit(in_len, SKGR, 1)
polynomial_SKGR = np.poly1d(coefficients_SKGR)
equation_SKGR = f'y = {coefficients_SKGR[0]:.2f}x + {coefficients_SKGR[1]:.2f}'
ax[1, 1].text(0.05, 0.95, equation_SKGR, transform=ax[1, 1].transAxes, fontsize=10, verticalalignment='top')
ax[1, 1].set_xlabel('Length of initial key')
ax[1, 1].set_ylabel('Secure key generation rate')
ax[1, 1].set_title("Input key length vs SKGR")
ax[1, 1].plot(in_len, SKGR)
ax[1, 1].plot(in_len, polynomial_SKGR(in_len), linestyle='--')
ax[1, 1].minorticks_on()
ax[1, 1].grid(True)

# Set the super title for the entire figure
plt.suptitle(title, weight='bold')

# Display the plots
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

# Print the title
print(title)


In [ ]:
if write :
    filename = f"Data.txt"
    key_strings = f"Keys.txt"
    data_path = os.path.join(data_subdirectory, filename)
    key_path = os.path.join(data_subdirectory, key_strings)
    file = open(data_path, "a")
    file2 = open(key_path, "a")
    
    file.write(title)
    file2.write(title)
    
    print("Files updated")

    file.write(f"\nInput Length = [{', '.join(map(str, in_len))}] \nAverage Time Taken = [{', '.join(map(str, avg_time_taken))}] \nAverage QBER = [{', '.join(map(str, avg_QBERs))}] \nAverage Output length = [{', '.join(map(str, avg_out_len))}]\n \nSKGR = [{', '.join(map(str, SKGR))}]")
    file.close()
    
    file2.write(detection)
    #file2.write(" False detection indices : ", False_detections)
    file2.write(f"\nKeys = [{', '.join(map(str, keys))}] \n\n Eve detection = [{', '.join(map(str, Eve_detected))}]")
    file2.close()

In [ ]:
plt.plot(in_len, avg_out_len, label = "Output length")
plt.plot(in_len, avg_QBERs, label = "QBERs")
plt.plot(in_len, avg_time_taken, label = "time_taken")
plt.plot(in_len, SKGR, label = 'SKGR')


plt.legend()

In [ ]:
time_taken = np.reshape(time_taken, (20,))  # Assuming time_taken has 20 elements

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot input length vs SKGR
color = 'tab:red'
ax1.set_xlabel('Length of initial key')
ax1.set_ylabel('Secure Key Generation Rate (SKGR)', color=color)
ax1.plot(in_len, SKGR, label='SKGR', color=color)
ax1.tick_params(axis='y', labelcolor=color)

# Create a secondary x-axis on the top for output length
ax2 = ax1.twiny()
color = 'tab:blue'
ax2.set_xlabel('Length of final key after Information Reconciliation', color=color)
ax2.plot(in_len, avg_out_len, label='Output Length', color=color)
ax2.tick_params(axis='x', labelcolor=color)

# Synchronize the x-axis for input length and output length
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(in_len)
ax2.set_xticklabels(avg_out_len)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Set the title
plt.title(title, weight='bold')

# Display the plot
plt.tight_layout()
plt.show()

# Print the titleimport matplotlib.pyplot as plt

fig, ax1 = plt.subplots(figsize=(10, 6))

# Plot input length vs SKGR
color_skgr = 'tab:red'
ax1.set_xlabel('Length of initial key')
ax1.set_ylabel('Secure Key Generation Rate (SKGR)', color=color_skgr)
ax1.plot(in_len, SKGR, label='SKGR', color=color_skgr)
ax1.tick_params(axis='y', labelcolor=color_skgr)

# Create a secondary y-axis on the right for time taken
ax2 = ax1.twinx()
color_time = 'tab:blue'
ax2.set_ylabel('Time Taken (seconds)', color=color_time)
ax2.plot(in_len, time_taken, label='Time Taken', color=color_time)  # Replace 'time_taken' with your data
ax2.tick_params(axis='y', labelcolor=color_time)

# Set limits for SKGR (0 to 0.5)
ax1.set_ylim(0, 0.5)

# Set limits for time taken (0 to 50)
ax2.set_ylim(0, 50)

# Synchronize the x-axis for input length
ax2.set_xticks(in_len)
ax2.set_xticklabels(in_len)  # Use in_len for labels as well (assuming it represents input length)

# Add legends
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

# Set the title
plt.title(title, weight='bold')

# Display the plot
plt.tight_layout()
plt.show()

# Print the title
print(title)

print(title)


In [ ]:
ifig, ax1 = plt.subplots(figsize=(10, 6))

# Plot input length vs SKGR on primary x and y axes
color_skgr = 'tab:red'
ax1.set_xlabel('Input Length')
ax1.set_ylabel('SKGR', color=color_skgr)
ax1.tick_params(axis='y', labelcolor=color_skgr)
ax1.plot(in_len, SKGR, label='SKGR', color=color_skgr)

# Create a secondary x-axis on top for output length (scaled ticks)
ax2 = ax1.twinx()
color_out_len = 'tab:green'
ax2.set_xlabel('Output Length (linearly related to Input Length)')
ax2.set_ylabel('Normalized Output Length', color=color_out_len)
ax2.tick_params(axis='y', labelcolor=color_out_len)

# Calculate normalized output length (assuming linear relationship with input length)
normalized_out_len = (avg_out_len - min(avg_out_len)) / (max(avg_out_len) - min(avg_out_len))

# Plot normalized output length on the secondary x-axis with scaled ticks
max_out_len = max(normalized_out_len)
ax2.set_ylim(0, max_out_len * 1.1)  # Adjust y-axis limit slightly for better visualization
ax2.set_xticks(in_len)
ax2.set_xticklabels(normalized_out_len, color=color_out_len)  # Use normalized values for labels

# Create a secondary y-axis on the right for time taken
ax3 = ax1.twinx()
color_time = 'tab:blue'
ax3.set_ylabel('Time Taken (seconds)', color=color_time)
ax3.tick_params(axis='y', labelcolor=color_time)
ax3.plot(in_len, time_taken, label='Time Taken', color=color_time)

# Set limits for SKGR (0 to 0.5) and time taken (adjust as needed)
ax1.set_ylim(0, 0.5)  # Assuming SKGR ranges from 0 to 0.5

"""# Linear regression for all data (optional)
slope_skgr, intercept_skgr = np.polyfit(in_len, SKGR, 1)  # Fit for SKGR
slope_time, intercept_time = np.polyfit(in_len, time_taken, 1)  # Fit for time taken

# Plot linear fits (optional)
x_fit = np.linspace(min(in_len), max(in_len), 100)  # Range for fit lines
ax1.plot(x_fit, slope_skgr * x_fit + intercept_skgr, color='gray', linestyle='dashed', label='SKGR Fit')
ax3.plot(x_fit, slope_time * x_fit + intercept_time, color='gray', linestyle='dashed', label='Time Taken Fit')
"""
# Add legends
handles1, labels1 = ax1.get_legend_handles_labels()
handles2, labels2 = ax2.get_legend_handles_labels()
handles3, labels3 = ax3.get_legend_handles_labels()
ax1.legend(handles1 + handles2 + handles3, labels1 + labels2 + labels3, loc='upper left')

# Set the title
plt.title(title, weight='bold')

# Display the plot
plt.tight_layout()
plt.show()

# Print the title
print(title)


In [ ]:
import seaborn as sns

sns.lineplot(in_len, avg_QBERs, hue = )